# Fashion MNIST Neural Network

This notebook implements a neural network for the Fashion MNIST dataset with improvements over the original MNIST implementation:
1. Uses Fashion MNIST instead of MNIST
2. Adds batch normalization for better training stability
3. Implements dropout for regularization
4. Uses ReLU activation function
5. Adds a second hidden layer
6. Implements early stopping

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
# Load and preprocess the Fashion MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize pixel values to range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Print the shapes of our datasets
print('Training Data Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Data Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

In [ ]:
# Define class names for Fashion MNIST
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Display sample images
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i]])
plt.show()

In [ ]:
# Reshape the data for the neural network
X_train_reshaped = X_train.reshape((X_train.shape[0], 28*28))
X_test_reshaped = X_test.reshape((X_test.shape[0], 28*28))

# Convert labels to one-hot encoded format
y_train_encoded = keras.utils.to_categorical(y_train)
y_test_encoded = keras.utils.to_categorical(y_test)

In [ ]:
# Build the improved neural network model
model = keras.Sequential([
    # Input layer
    layers.Input(shape=(784,)),
    
    # First hidden layer with batch normalization and dropout
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    # Second hidden layer with batch normalization and dropout
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    # Output layer
    layers.Dense(10, activation='softmax')
])

# Display model summary
model.summary()

In [ ]:
# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Add early stopping callback
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train_reshaped,
    y_train_encoded,
    epochs=50,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training & validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_encoded, verbose=0)
print(f'Test accuracy: {test_accuracy:.4f}')

# Make predictions
predictions = model.predict(X_test_reshaped)
predicted_classes = np.argmax(predictions, axis=1)

# Plot confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# Display some example predictions
plt.figure(figsize=(15,6))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_test[i], cmap=plt.cm.binary)
    predicted_label = class_names[predicted_classes[i]]
    true_label = class_names[y_test[i]]
    color = 'green' if predicted_label == true_label else 'red'
    plt.xlabel(f'{predicted_label}\n({true_label})', color=color)
plt.show()

## Improvements Over Original MNIST Implementation

1. **Architecture Improvements**:
   - Added a second hidden layer (512 -> 256 -> 10)
   - Implemented batch normalization for better training stability
   - Added dropout layers (0.3) to prevent overfitting
   - Used ReLU activation for better gradient flow

2. **Training Improvements**:
   - Implemented early stopping to prevent overfitting
   - Used Adam optimizer with learning rate 0.001
   - Added validation split for monitoring training

3. **Visualization Improvements**:
   - Added confusion matrix visualization
   - Improved plotting of training history
   - Added example predictions with color-coded results

The Fashion MNIST dataset is more challenging than the original MNIST, but these improvements help achieve good accuracy while preventing overfitting.